In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from datetime import datetime
from packaging import version

2022-03-04 15:34:31.735518: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-04 15:34:31.735544: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Load data

In [2]:
cf15= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/cf15.csv', low_memory=False, header=None)
gf25= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/gf25.csv', low_memory=False, header=None)
gm01= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/gm01.csv', low_memory=False, header=None)
nc46= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/nc46.csv', low_memory=False, header=None)
nc74= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/nc74.csv', low_memory=False, header=None)
nf11= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/nf11.csv', low_memory=False, header=None)
no22= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/no22.csv', low_memory=False, header=None)
wc16= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/wc16.csv', low_memory=False, header=None)
wc41= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/wc41.csv', low_memory=False, header=None)
wo27= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/wo27.csv', low_memory=False, header=None)
wo34= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/wo34.csv', low_memory=False, header=None)
wo36= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/wo36.csv', low_memory=False, header=None)
wo37= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/wo37.csv', low_memory=False, header=None)
wo40= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/wo40.csv', low_memory=False, header=None)
wo45= pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/wo45.csv', low_memory=False, header=None)

In [3]:
val_1 = pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/gm01_EarlyMarch.csv', low_memory=False, header=None)
val_2 = pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/wo36_EarlyMarch.csv', low_memory=False, header=None)
val_3 = pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/go22.csv', low_memory=False, header=None)

In [4]:
frames =[cf15, gf25,gm01,nc46, nc74,nf11, wc16, wc41,wo27,wo34, wo36, wo37, wo40, wo45]
frames_2=[val_1, val_2, val_3]
data = pd.concat(frames)
val = pd.concat(frames_2)

Define data cleaning functions

In [5]:
def snip(data):
    #Snip unwanted characters 

    time= data.iloc[:,0]
    time=time.map(lambda x: str(x[6:]))
    time=time.map(lambda x: float(x))

    interface= data.iloc[:,-14]
    interface=interface.map(lambda x: str(x))
    interface=interface.map(lambda x: str(x[15:]))
    interface=interface.map(lambda x: str(x[:-1]))

    jitter=data.iloc[:, -13]
    jitter=jitter.map(lambda x: str(x))
    jitter=jitter.map(lambda x: str(x[8:]))
    jitter=jitter.map(lambda x: str(x[:-1]))
    jitter=jitter.map(lambda x: float(x))

    latency= data.iloc[:,-12]
    latency=latency.map(lambda x: str(x))
    latency=latency.map(lambda x: str(x[9:]))
    latency=latency.map(lambda x: str(x[:-1]))
    latency=latency.map(lambda x: float(x))

    inband= data.iloc[:,-15]
    inband=inband.map(lambda x: str(x))
    inband=inband.map(lambda x: str(x[22:]))
    inband=inband.map(lambda x: str(x[:-5]))
    inband=inband.map(lambda x: float(x))

    outband= data.iloc[:, -5]
    outband=outband.map(lambda x: str(x))
    outband=outband.map(lambda x: x.split('"')[1])
    outband=outband.map(lambda x: str(x[:-4]))
    outband=outband.map(lambda x: float(x))

    packet=data.iloc[:,-4]
    packet=packet.map(lambda x: str(x))
    packet=packet.map(lambda x: str(x[12:]))
    packet=packet.map(lambda x: str(x[:-2]))
    packet=packet.map(lambda x: float(x))

    status=data.iloc[:,-2]
    status=status.map(lambda x: str(x))
    status=status.map(lambda x: str(x[8:]))
    status=status.map(lambda x: str(x[:-1]))

    status[:] = np.where(status=="up", 1,0)
    status=status.map(lambda x: float(x))
    #status=status.rolling(20).mean()
    #status=status.map(np.floor)

    train_init=pd.concat([time, interface, inband, outband, latency, packet, jitter, status], axis=1)
    train_init.columns=['Time', 'Interface','Inband', 'Outband', 'Latency', 'Packet', 'Jitter', 'Status']

    #Dedupp based on interfaces
    train=train_init[train_init.Interface != 'NI2A-WWW1']
    train=train[train.Interface != 'NI2B-WWW1']
    train=train[train.Interface != 'NI2A-WWW2']
    train=train[train.Interface != 'NI2B-WWW2']
    train=train[train.Interface != 'NI2A-WWW3']    
    train=train[train.Interface != 'NI2B-WWW3']
    train=train[train.Interface != 'NI2A-MPLS']
    train['Interface']= train['Interface'].astype('category')
    train['Interface']=train['Interface'].cat.codes

    train_label=train.filter(['Status'])
    train=train.drop(['Status', 'Inband', 'Outband'], axis=1)
    train=train.reset_index(drop=True)

    return train, train_label

In [6]:
def preproccess(data):
    train,train_label=snip(data)
    train=train.iloc[:-700, :]
    train=train.to_numpy()
    train_label=train_label.iloc[700:]
    return train, train_label


In [7]:
train, train_label = preproccess(data)
mean=train.mean(axis=0)
train-=mean
std = train.std(axis=0)
train/=std
test, test_label = preproccess(val)
test-=mean
test/=std

train_dataset = tf.data.Dataset.from_tensor_slices((train, train_label))
test_dataset = tf.data.Dataset.from_tensor_slices((test, test_label))

In [8]:
logs = "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520')

2022-03-04 15:35:15.271187: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2022-03-04 15:35:15.271263: I tensorflow/core/profiler/lib/profiler_session.cc:125] Profiler session started.
2022-03-04 15:35:17.236709: I tensorflow/core/profiler/lib/profiler_session.cc:143] Profiler session tear down.


Build and train model

In [9]:
def get_model():
    model = tf.keras.Sequential([
            tf.keras.layers.Dense(128, activation='relu',kernel_regularizer='l1_l2'),
            #tf.keras.layers.Dropout(0.4),
            tf.keras.layers.Dense(128, activation='relu',kernel_regularizer='l1_l2'),
            #tf.keras.layers.Dropout(0.4),
            #tf.keras.layers.Dense(16, activation='relu', kernel_regularizer='l1_l2'),
            #tf.keras.layers.Dropout(0.3),
            #tf.keras.layers.Dense(10, activation='relu', kernel_regularizer='l1_l2'),
            #tf.keras.layers.Dropout(0.3),
            #tf.keras.layers.Dense(10, activation='relu', kernel_regularizer='l1_l2'),
            #tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(1, activation='sigmoid')])
    model.compile(optimizer='adam',
            loss='BinaryCrossentropy',
            metrics=['accuracy'])    
    return model

In [10]:
model = get_model()       
history = model.fit(train, train_label, epochs=6, batch_size=128,  callbacks = [tboard_callback])

Epoch 1/6


2022-03-04 15:35:19.064103: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-03-04 15:35:19.064153: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pop-os): /proc/driver/nvidia/version does not exist
2022-03-04 15:35:19.071761: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


 540/5394 [==>...........................] - ETA: 12s - loss: 1.9561 - accuracy: 0.7999

2022-03-04 15:35:21.211880: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2022-03-04 15:35:21.211927: I tensorflow/core/profiler/lib/profiler_session.cc:125] Profiler session started.
2022-03-04 15:35:21.269075: I tensorflow/core/profiler/lib/profiler_session.cc:67] Profiler session collecting data.
2022-03-04 15:35:21.278036: I tensorflow/core/profiler/lib/profiler_session.cc:143] Profiler session tear down.
2022-03-04 15:35:21.296423: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/20220304-153515/plugins/profile/2022_03_04_15_35_21

2022-03-04 15:35:21.308331: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to logs/20220304-153515/plugins/profile/2022_03_04_15_35_21/pop-os.trace.json.gz
2022-03-04 15:35:21.322634: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/20220304-153515/plugins/profile/2022_03_04_15_35_21

2022-

5394/5394 [==============================] - 14s 3ms/step - loss: 0.6618 - accuracy: 0.8021
Epoch 2/6
5394/5394 [==============================] - 15s 3ms/step - loss: 0.5183 - accuracy: 0.8021
Epoch 3/6
5394/5394 [==============================] - 15s 3ms/step - loss: 0.5182 - accuracy: 0.8021
Epoch 4/6
5394/5394 [==============================] - 18s 3ms/step - loss: 0.5182 - accuracy: 0.8021
Epoch 5/6
5394/5394 [==============================] - 17s 3ms/step - loss: 0.5184 - accuracy: 0.8021
Epoch 6/6
5394/5394 [==============================] - 16s 3ms/step - loss: 0.5184 - accuracy: 0.8021


In [13]:
test_loss, test_acc =model.evaluate(test, test_label, verbose=2)
print('\nTest Accuracy:', test_acc)

2910/2910 - 3s - loss: 0.2972 - accuracy: 0.9604 - 3s/epoch - 861us/step

Test Accuracy: 0.9603900909423828


In [12]:
model.save('model.h5')